In [4]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [5]:
tf.version.VERSION

'2.9.2'

In [36]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [37]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


2022-09-14 21:14:35.492104: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [13]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

b'You So Crazy (1994)'


2022-09-14 16:24:27.387887: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [38]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [39]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}


2022-09-14 21:20:24.435199: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [32]:
for x in train.take(3).as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b'Postman, The (1997)', 'user_id': b'681'}
{'movie_title': b'Clueless (1995)', 'user_id': b'442'}
{'movie_title': b'Maltese Falcon, The (1941)', 'user_id': b'932'}


In [16]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [41]:
movie_titles

<BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [40]:
type(movies)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [35]:
type(unique_user_ids[0])

bytes

In [17]:
embedding_dimension = 32

In [18]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [19]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [42]:
type(movies)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [20]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [21]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [22]:
class MovielensModel(tfrs.Model):

    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.movie_model(features["movie_title"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

In [23]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [24]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [25]:
model.fit(cached_train, epochs=3)

Epoch 1/3


10/10 [==============================] - 4s 333ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0097 - factorized_top_k/top_10_categorical_accuracy: 0.0205 - factorized_top_k/top_50_categorical_accuracy: 0.1004 - factorized_top_k/top_100_categorical_accuracy: 0.1774 - loss: 69885.1094 - regularization_loss: 0.0000e+00 - total_loss: 69885.1094
Epoch 2/3
10/10 [==============================] - 3s 319ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.0187 - factorized_top_k/top_10_categorical_accuracy: 0.0378 - factorized_top_k/top_50_categorical_accuracy: 0.1696 - factorized_top_k/top_100_categorical_accuracy: 0.2929 - loss: 67523.3693 - regularization_loss: 0.0000e+00 - total_loss: 67523.3693
Epoch 3/3
10/10 [==============================] - 3s 319ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0225 - f

In [26]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 126ms/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0098 - factorized_top_k/top_10_categorical_accuracy: 0.0222 - factorized_top_k/top_50_categorical_accuracy: 0.1246 - factorized_top_k/top_100_categorical_accuracy: 0.2325 - loss: 31079.0635 - regularization_loss: 0.0000e+00 - total_loss: 31079.0635


{'factorized_top_k/top_1_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_5_categorical_accuracy': 0.009800000116229057,
 'factorized_top_k/top_10_categorical_accuracy': 0.02215000055730343,
 'factorized_top_k/top_50_categorical_accuracy': 0.12460000067949295,
 'factorized_top_k/top_100_categorical_accuracy': 0.2325499951839447,
 'loss': 28244.771484375,
 'regularization_loss': 0,
 'total_loss': 28244.771484375}

In [44]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# recommends movies out of the entire movies dataset.
index.index(movies.batch(100).map(model.movie_model), movies)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :5]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)'
 b'101 Dalmatians (1996)' b'While You Were Sleeping (1995)']
